## 01 Data loading and cleaning

In [1]:
# the imports
import pandas as pd
import numpy as np

In [2]:
#loading the data
df = pd.read_csv('../data/final.csv')
df_cops = pd.read_csv('../data/2010_police_num_by_county.csv')
df_people = pd.read_csv('../data/people.csv')

pd.set_option('display.max_columns', 35)

In [3]:
#checking on the data amount
df.shape

(3150, 23)

In [4]:
#checking on the data amount
df_cops.shape

(3034, 4)

In [5]:
#checking on the data amount
df_people.shape

(3212, 7)

In [6]:
df[df['fips'].isna()]

,state,fips,county_name,unemployment_rate_2010,population_total_2010,urban_population_prc,rural_population_prc,crime_per_capita,per_capita_sme_num,per_capita_large_num,avg_ann_pay_per_emp_sme,avg_ann_pay_per_emp_large,avg_ann_pay_per_emp_total,population_jail_prc,2008_dem_%_vote,2008_rep_%_vote,2008_other_%_vote,smoke_percent_2010,popul_hs_grad_prc,popul_college_grad_prc,popul_single_paren_prc,liquor_stores_per10k,foreign_key
72,AK,NaN,Chugach,NaN,NaN,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0,0,0,NaN,NaN,NaN,NaN,NaN,Alaska Chugach
73,AK,NaN,Copper River,NaN,NaN,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0,0,0,NaN,NaN,NaN,NaN,NaN,Alaska Copper River
1603,NaN,NaN,NaN,NaN,NaN,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.drop([72, 73, 1603], axis=0, inplace=True)

In [8]:
#merging the databases together and cleaning the NA values
df = pd.merge(df, df_cops, how='left', left_on='foreign_key', right_on='foreign_police')

df = pd.merge(df, df_people, how='left', left_on='fips', right_on='FIPS_people')

df.shape

(3147, 34)

In [9]:
#dropping unnecessary columns from dataframe merging
df = df.drop(columns=['fips', 'foreign_key', 'state_police', 'county_police', 'foreign_police', 'FIPS_people'])

In [10]:
#cleaning the data of unnecessary symbols
j=[]
for i in df['population_total_2010']:
    if not type(i) == float:
        j.append(i.replace(',', ''))
    else:
        j.append(i)
df['population_total_2010'] = j

k=[]
for m in df['number_police']:
    if not type(m) == float:
        k.append(m.replace(',', ''))
    else:
        k.append(m)
df['number_police'] = k

In [11]:
#cleaning the data of non numeric values
cols = ['avg_ann_pay_per_emp_large', 'avg_ann_pay_per_emp_sme', 'avg_ann_pay_per_emp_total', '2008_dem_%_vote', '2008_rep_%_vote', '2008_other_%_vote']
for i in cols:
    df[i] = [txt if txt != '#DIV/0!' else np.nan for txt in df[i]]

In [12]:
#changing the datatypes
col_names = list(df.columns)
col_names = [i for i in col_names if not i in ['state', 'county_name']]

for b in col_names:
    try:
        df[b] = df[b].astype(float)
    except:
        pass

In [13]:
cols = ['urban_population_prc', 'rural_population_prc', 'crime_per_capita', 'per_capita_sme_num', 'per_capita_large_num', 'population_jail_prc']
for col in cols:
    df[col] = [txt if txt != '#DIV/0!' else np.nan for txt in df[col]]
for i in list(df.columns):
    try:
        df[i] = df[i].astype(float)
    except:
        pass

In [14]:
df.dtypes

state                                object
county_name                          object
unemployment_rate_2010              float64
population_total_2010               float64
urban_population_prc                float64
rural_population_prc                float64
crime_per_capita                    float64
per_capita_sme_num                  float64
per_capita_large_num                float64
avg_ann_pay_per_emp_sme             float64
avg_ann_pay_per_emp_large           float64
avg_ann_pay_per_emp_total           float64
population_jail_prc                 float64
2008_dem_%_vote                     float64
2008_rep_%_vote                     float64
2008_other_%_vote                   float64
smoke_percent_2010                  float64
popul_hs_grad_prc                   float64
popul_college_grad_prc              float64
popul_single_paren_prc              float64
liquor_stores_per10k                float64
number_police                       float64
WhiteNonHispanicPct2010         

In [15]:
#derriving average values by state to fill in the NA cells
avg_val = df.groupby('state').mean()

for i in list(df.columns):
    df[i] = df[i].astype(str)

In [16]:
avg_val.head(2)

,unemployment_rate_2010,population_total_2010,urban_population_prc,rural_population_prc,crime_per_capita,per_capita_sme_num,per_capita_large_num,avg_ann_pay_per_emp_sme,avg_ann_pay_per_emp_large,avg_ann_pay_per_emp_total,population_jail_prc,2008_dem_%_vote,2008_rep_%_vote,2008_other_%_vote,smoke_percent_2010,popul_hs_grad_prc,popul_college_grad_prc,popul_single_paren_prc,liquor_stores_per10k,number_police,WhiteNonHispanicPct2010,BlackNonHispanicPct2010,AsianNonHispanicPct2010,NativeAmericanNonHispanicPct2010,HispanicPct2010,MultipleRacePct2010
state,,,,,,,,,,,,,,,,,,,,,,,,,,
AK,10.762069,24490.724138,0.235056,0.804550,0.004191,0.021140,0.001848,65273.797475,58940.338911,49221.093286,0.000622,0.000000,0.000000,0.000000,27.530526,63.923077,18.555556,12.518519,2.885185,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL,12.150746,71339.343284,0.329139,0.670861,0.002948,0.014109,0.001842,28469.141502,36807.208207,32008.140105,0.003516,0.382693,0.608365,0.008943,26.353731,65.343284,14.880597,10.462687,0.701493,87.25,66.10194,28.250896,0.571343,0.639552,3.190299,1.34403


In [17]:
#filling in the NA cells
for i in list(df.columns):
    for j in list(df.index):
        if df.loc[j,i] == 'nan':
            state = df.loc[j,:]['state']
            var = avg_val[avg_val.index == state][i]
            df.loc[j,i] = var[0]

In [18]:
df.shape

(3147, 28)

In [19]:
#cleaning more NAs
df = df.dropna()

In [21]:
for col in list(df.columns):
    try:
        df[col] = df[col].astype(float)
    except:
        pass

In [22]:
df.dtypes

state                                object
county_name                          object
unemployment_rate_2010              float64
population_total_2010               float64
urban_population_prc                float64
rural_population_prc                float64
crime_per_capita                    float64
per_capita_sme_num                  float64
per_capita_large_num                float64
avg_ann_pay_per_emp_sme             float64
avg_ann_pay_per_emp_large           float64
avg_ann_pay_per_emp_total           float64
population_jail_prc                 float64
2008_dem_%_vote                     float64
2008_rep_%_vote                     float64
2008_other_%_vote                   float64
smoke_percent_2010                  float64
popul_hs_grad_prc                   float64
popul_college_grad_prc              float64
popul_single_paren_prc              float64
liquor_stores_per10k                float64
number_police                       float64
WhiteNonHispanicPct2010         

In [23]:
#creating a per_capita_police columns out of population totals and number of police
per_capita_police = df['number_police'] / df['population_total_2010']

police_1000 = per_capita_police * 1000

df = df.rename(columns={'number_police': 'police_per_1000'})

df['police_per_1000'] = police_1000

In [24]:
for i in ['smoke_percent_2010', 'popul_hs_grad_prc', 'popul_college_grad_prc', 'popul_single_paren_prc']:
    df[i] = df[i] / 100

In [25]:
df.head(2)

,state,county_name,unemployment_rate_2010,population_total_2010,urban_population_prc,rural_population_prc,crime_per_capita,per_capita_sme_num,per_capita_large_num,avg_ann_pay_per_emp_sme,avg_ann_pay_per_emp_large,avg_ann_pay_per_emp_total,population_jail_prc,2008_dem_%_vote,2008_rep_%_vote,2008_other_%_vote,smoke_percent_2010,popul_hs_grad_prc,popul_college_grad_prc,popul_single_paren_prc,liquor_stores_per10k,police_per_1000,WhiteNonHispanicPct2010,BlackNonHispanicPct2010,AsianNonHispanicPct2010,NativeAmericanNonHispanicPct2010,HispanicPct2010,MultipleRacePct2010
0,AL,Autauga,8.8,54571.0,0.579978,0.420022,0.002620,0.012259,0.002565,22667.03037,31792.20223,26856.69322,0.002914,0.257730,0.736136,0.006133,0.2577,0.72,0.21,0.09,0.4,1.172784,77.25,17.58,0.86,0.40,2.40,1.59
1,AL,Baldwin,9.9,182265.0,0.577209,0.422791,0.002112,0.020108,0.002019,28664.84962,29390.96748,28943.22956,0.004030,0.238119,0.752595,0.009286,0.2297,0.67,0.26,0.09,0.8,1.486846,83.50,9.31,0.74,0.63,4.38,1.49


In [26]:
df.shape

(3015, 28)

In [27]:
#saving the data
df.to_csv('../data/final_work_data.csv', index=False)